In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
carpark = pd.read_csv('carpark.csv')

In [3]:
info = pd.read_csv('hdb-carpark-information.csv')
info = info[info.car_park_no.isin(carpark.ID.unique())]

In [4]:
carparkx = pd.DataFrame()
carparkx['id'] = carpark.ID.unique()
carparkx.index = carparkx['id']
for i, row in carpark.iterrows():
    if row.Date not in carparkx.columns:
        carparkx[row.Date] = None
    carparkx.loc[row.ID, row.Date] = row.Available

KeyboardInterrupt: 

In [4]:
def nearest5(id):
    curr_loc_x = info[info.car_park_no == id].x_coord.iloc[0]
    curr_loc_y = info[info.car_park_no == id].y_coord.iloc[0]
    info['dist_away_x'] = info.x_coord - curr_loc_x
    info['dist_away_y'] = info.y_coord - curr_loc_y
    info['dist_away'] = info['dist_away_x']**2 + info['dist_away_y'] ** 2
    info['dist_away'] = info['dist_away'] ** 0.5
    return list(info.sort_values('dist_away').car_park_no.iloc[:5])

In [20]:
carpark_train = carpark[carpark.Date > '2019-09-14T18:00:00']
carpark.index= carpark.ID + carpark.Date

In [21]:
train_df = pd.DataFrame()
train_df['target'] = carpark_train.Available/carpark_train.Total
train_df['date'] = carpark_train.Date
train_df['id'] = carpark_train.ID
train_df.index  = train_df.id + train_df.date

In [22]:
datetime_object = datetime.strptime('2019-09-08T18:00:00', '%Y-%m-%dT%H:%M:%S')

In [23]:
from tqdm import tqdm

In [41]:
row.name

'TG22019-09-15T18:00:00'

In [50]:
for x in ['curr_0', 'curr_1', 'curr_2', 'curr_3', 'curr_4', 'prev_0', 'prev_1', 'prev_2', 'prev_3', 'prev_4']: train_df[x] = None
for id, row in tqdm(train_df.iloc[:1000].iterrows(), total=train_df.iloc[:1000].shape[0]):
    try:
        curr_id = row.id
        near5 = nearest5(curr_id)
        curr_time = (datetime.strptime(row.date, '%Y-%m-%dT%H:%M:%S') - timedelta(hours = 1)).strftime('%Y-%m-%dT%H:%M:%S')
        prev_time = (datetime.strptime(row.date, '%Y-%m-%dT%H:%M:%S') - timedelta(days = 7)).strftime('%Y-%m-%dT%H:%M:%S')
        near5_curr = [near + curr_time for near in near5]
        near5_prev = [near + prev_time for near in near5]
        [train_df.loc[row.name, ['curr_0', 'curr_1', 'curr_2', 'curr_3', 'curr_4']] = list(carpark.loc[near5_curr, :].Available / carpark.loc[near5_curr, :].Total)
        train_df.loc[row.name, ['prev_0', 'prev_1', 'prev_2', 'prev_3', 'prev_4']] = list(carpark.loc[near5_prev, :].Available / carpark.loc[near5_prev, :].Total)
]    except:
        continue

100%|██████████| 1000/1000 [03:25<00:00,  6.93it/s]


In [45]:
list(carpark.loc[near5_curr, :].Available / carpark.loc[near5_curr, :].Total)

[0.4065934065934066,
 0.6507936507936508,
 0.6984536082474226,
 0.7525773195876289,
 0.5432692307692307]

In [75]:
final_train = train_df[~train_df.isnull().apply(lambda row: any(row), axis = 1)]

In [76]:
final_train[['target', 'curr_0', 'curr_1', 'curr_2', 'curr_3', 'curr_4', 'prev_0', 'prev_1', 'prev_2', 'prev_3', 'prev_4']] = final_train[['target','curr_0', 'curr_1', 'curr_2', 'curr_3', 'curr_4', 'prev_0', 'prev_1', 'prev_2', 'prev_3', 'prev_4']].astype(np.float)

/Users/vincent/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [68]:
import xgboost as xgb
from sklearn.model_selection import KFold

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_train[['curr_0', 'curr_1', 'curr_2', 'curr_3', 'curr_4', 'prev_0', 'prev_1', 'prev_2', 'prev_3', 'prev_4']], final_train.target, test_size=0.2, random_state=123)

In [79]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [87]:
xg_reg.fit(X_train.values ,y_train.values)

preds = xg_reg.predict(X_test.values)

[19:08:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [88]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.162603


In [90]:
import pickle
file_name = "xgb_slot.pkl"

# save
pickle.dump(xg_reg, open(file_name, "wb"))

# load
xgb_model_loaded = pickle.load(open(file_name, "rb"))


xgb_model_loaded.predict(X_test.values)[0] == preds[0]

[19:09:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


True